In [12]:
print(common_rows)

                                   query total_clicks total_impressions  \
0                          brokerchooser       42,766           390,355   
1                     trading 212 review       11,285           241,417   
2                    interactive brokers        9,059           708,432   
3                                revolut        8,447           596,897   
4                         broker chooser        8,372            75,277   
...                                  ...          ...               ...   
8635              is kot4x a good broker            2                77   
8644  does fidelity charge a monthly fee            2                19   
8647            routing number 042200910            2                 4   
8656                  how safe is etrade            2               154   
8661      is sofi a safe place to invest            2                44   

                                                top_url       content_type  
0                     

In [13]:
import pandas as pd

# Load the 2 CSV files
jan_df = pd.read_csv('data/usa-january.csv')
feb_df = pd.read_csv('data/usa-february.csv')
# csv_file1 = pd.read_csv('data/gsc-with-categories-january.csv')
# csv_file2 = pd.read_csv('data/gsc-with-categories-february.csv')

# Get a boolean value whether each row is in the column of another DataFrame
is_in_both = jan_df['query'].isin(feb_df['query'])

# Get rows which are in both files
common_rows = csv_file1[is_in_both]

# Export to CSV
# common_rows.to_csv('data/usa-january-february-common.csv', index=False)


In [17]:
import pandas as pd

# Load the CSV files and convert numeric columns
# jan_df = pd.read_csv('january_data.csv')
# feb_df = pd.read_csv('february_data.csv')

# Convert string columns to numeric
numeric_columns = ['total_clicks', 'total_impressions']
jan_df[numeric_columns] = jan_df[numeric_columns].replace({',': ''}, regex=True).apply(pd.to_numeric, errors='coerce')
feb_df[numeric_columns] = feb_df[numeric_columns].replace({',': ''}, regex=True).apply(pd.to_numeric, errors='coerce')



# Assuming common_rows is already created and contains the common queries
# If you need to create common_rows:
# common_rows = pd.DataFrame({'query': list(set(jan_df['query']).intersection(set(feb_df['query'])))})

# Create a new DataFrame to store the results
result_df = pd.DataFrame()

# Add the query column from common_rows
result_df['query'] = common_rows['query']

# For each common query, find the data in both January and February
for index, row in result_df.iterrows():
    query = row['query']
    
    # Get the January and February data for this query
    jan_row = jan_df.loc[jan_df['query'] == query].iloc[0]
    feb_row = feb_df.loc[feb_df['query'] == query].iloc[0]
    
    # Calculate differences
    click_diff = feb_row['total_clicks'] - jan_row['total_clicks']
    impression_diff = feb_row['total_impressions'] - jan_row['total_impressions']
    
    # Use January's top_url and content_type
    top_url = jan_row['top_url']
    content_type = jan_row['content_type']
    
    # Add to result DataFrame
    result_df.at[index, 'jan_clicks'] = jan_row['total_clicks']
    result_df.at[index, 'feb_clicks'] = feb_row['total_clicks']
    result_df.at[index, 'click_diff'] = click_diff
    result_df.at[index, 'click_pct_change'] = (click_diff / jan_row['total_clicks'] * 100) if jan_row['total_clicks'] > 0 else None
    
    result_df.at[index, 'jan_impressions'] = jan_row['total_impressions']
    result_df.at[index, 'feb_impressions'] = feb_row['total_impressions']
    result_df.at[index, 'impression_diff'] = impression_diff
    result_df.at[index, 'impression_pct_change'] = (impression_diff / jan_row['total_impressions'] * 100) if jan_row['total_impressions'] > 0 else None
    
    result_df.at[index, 'top_url'] = top_url
    result_df.at[index, 'content_type'] = content_type

# Sort by click difference in descending order
result_df = result_df.sort_values(by='click_diff', ascending=False)

# Save the results to a new CSV file
result_df.to_csv('data/query_comparison_jan_feb.csv', index=False)

print(f"Comparison completed. Found {len(result_df)} common queries.")
print("Top 5 queries with biggest click increases:")
print(result_df.head(5)[['query', 'click_diff', 'click_pct_change']])

Comparison completed. Found 3396 common queries.
Top 5 queries with biggest click increases:
                           query  click_diff  click_pct_change
0                  brokerchooser         0.0               0.0
4314           brokers in mexico         0.0               0.0
4318  when is google stock split         0.0               0.0
4322  where can you short stocks         0.0               0.0
4324          what is qfs ledger         0.0               0.0


In [7]:
print(common_rows)


                                   query total_clicks total_impressions  \
0                          brokerchooser       42,766           390,355   
1                     trading 212 review       11,285           241,417   
2                    interactive brokers        9,059           708,432   
3                                revolut        8,447           596,897   
4                         broker chooser        8,372            75,277   
...                                  ...          ...               ...   
8635              is kot4x a good broker            2                77   
8644  does fidelity charge a monthly fee            2                19   
8647            routing number 042200910            2                 4   
8656                  how safe is etrade            2               154   
8661      is sofi a safe place to invest            2                44   

                                                top_url       content_type  
0                     

In [20]:
import pandas as pd

# Load the CSV files
jan_df = pd.read_csv('data/category-gsc-january.csv')
feb_df = pd.read_csv('data/category-gsc-february.csv')

# Convert percentage strings to numeric values if needed
for df in [jan_df, feb_df]:
    if df['ctr_percentage'].dtype == 'object':
        df['ctr_percentage'] = df['ctr_percentage'].str.replace('%', '').astype(float)
    
    # Also handle numeric columns with commas if present
    for col in ['total_clicks', 'total_impressions', 'unique_urls']:
        if df[col].dtype == 'object':
            df[col] = df[col].str.replace(',', '').astype(float)

# Create a new dataframe with all unique content types
all_categories = pd.concat([jan_df['content_type'], feb_df['content_type']]).unique()
result_df = pd.DataFrame({'content_type': all_categories})

# Add January data
result_df = result_df.merge(
    jan_df[['content_type', 'total_clicks', 'total_impressions', 'ctr_percentage']], 
    on='content_type', 
    how='left',
    suffixes=('', '_jan')
)

# Rename columns for January
result_df.rename(columns={
    'total_clicks': 'jan_clicks',
    'total_impressions': 'jan_impressions',
    'ctr_percentage': 'jan_ctr'
}, inplace=True)

# Add February data
result_df = result_df.merge(
    feb_df[['content_type', 'total_clicks', 'total_impressions', 'ctr_percentage']], 
    on='content_type', 
    how='left',
    suffixes=('', '_feb')
)

# Rename columns for February
result_df.rename(columns={
    'total_clicks': 'feb_clicks',
    'total_impressions': 'feb_impressions',
    'ctr_percentage': 'feb_ctr'
}, inplace=True)

# Fill NaN values with 0 for categories that don't exist in one of the months
result_df = result_df.fillna(0)

# Calculate differences
result_df['clicks_diff'] = result_df['feb_clicks'] - result_df['jan_clicks']
result_df['impressions_diff'] = result_df['feb_impressions'] - result_df['jan_impressions']
result_df['ctr_diff'] = result_df['feb_ctr'] - result_df['jan_ctr']

# Calculate percentage changes
result_df['clicks_pct_change'] = result_df.apply(
    lambda x: ((x['feb_clicks'] - x['jan_clicks']) / x['jan_clicks'] * 100) if x['jan_clicks'] > 0 else float('inf'),
    axis=1
)
result_df['impressions_pct_change'] = result_df.apply(
    lambda x: ((x['feb_impressions'] - x['jan_impressions']) / x['jan_impressions'] * 100) if x['jan_impressions'] > 0 else float('inf'),
    axis=1
)

# Round all numeric columns to 2 decimal places
numeric_cols = result_df.select_dtypes(include=['float64', 'int64']).columns
result_df[numeric_cols] = result_df[numeric_cols].round(2)

# Replace infinity values with None for cleaner output
result_df.replace([float('inf'), -float('inf')], None, inplace=True)

# Sort by the absolute difference in clicks (highest impact first)
result_df = result_df.sort_values(by='clicks_diff', ascending=False)

# Save the result
result_df.to_csv('data/category_comparison_jan_feb.csv', index=False)

print(f"Analysis completed. Found {len(result_df)} unique content types.")
print("\nTop 5 categories with biggest click increases:")
print(result_df.head(5)[['content_type', 'clicks_diff', 'clicks_pct_change']])

print("\nTop 5 categories with biggest click decreases:")
print(result_df.sort_values(by='clicks_diff')[['content_type', 'clicks_diff', 'clicks_pct_change']].head(5))

Analysis completed. Found 11 unique content types.

Top 5 categories with biggest click increases:
                       content_type  clicks_diff  clicks_pct_change
9              Not_reviewed_brokers       1491.0             324.84
10                            Other          5.0               1.86
8   Company_name_options_megascaled       -251.0             -22.57
6                          Homepage       -528.0             -13.86
7                         Education       -664.0             -23.78

Top 5 categories with biggest click decreases:
        content_type  clicks_diff  clicks_pct_change
0      Broker_review     -28412.0             -24.05
1               Best     -25503.0             -29.62
2  Safety_megascaled      -9695.0             -11.64
4       Country_page      -6384.0             -31.54
3              Tools      -5188.0             -22.38


In [23]:
import pandas as pd

# Load the CSV files
jan_df = pd.read_csv('data/gsc-category-t9n-jan.csv')
feb_df = pd.read_csv('data/gsc-category-t9n-feb.csv')

# Convert percentage strings to numeric values if needed
for df in [jan_df, feb_df]:
    if df['ctr_percentage'].dtype == 'object':
        df['ctr_percentage'] = df['ctr_percentage'].str.replace('%', '').astype(float)
    
    # Also handle numeric columns with commas if present
    for col in ['total_clicks', 'total_impressions', 'unique_urls', 'avg_position']:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = df[col].str.replace(',', '').astype(float)

# Create a new dataframe with all unique content types
all_categories = pd.concat([jan_df['content_type'], feb_df['content_type']]).unique()
result_df = pd.DataFrame({'content_type': all_categories})

# Add January data
result_df = result_df.merge(
    jan_df[['content_type', 'total_clicks', 'total_impressions', 'ctr_percentage', 'avg_position']], 
    on='content_type', 
    how='left',
    suffixes=('', '_jan')
)

# Rename columns for January
result_df.rename(columns={
    'total_clicks': 'jan_clicks',
    'total_impressions': 'jan_impressions',
    'ctr_percentage': 'jan_ctr',
    'avg_position': 'jan_position'
}, inplace=True)

# Add February data
result_df = result_df.merge(
    feb_df[['content_type', 'total_clicks', 'total_impressions', 'ctr_percentage', 'avg_position']], 
    on='content_type', 
    how='left',
    suffixes=('', '_feb')
)

# Rename columns for February
result_df.rename(columns={
    'total_clicks': 'feb_clicks',
    'total_impressions': 'feb_impressions',
    'ctr_percentage': 'feb_ctr',
    'avg_position': 'feb_position'
}, inplace=True)

# Fill NaN values with 0 for categories that don't exist in one of the months
result_df = result_df.fillna(0)

# Calculate differences
result_df['clicks_diff'] = result_df['feb_clicks'] - result_df['jan_clicks']
result_df['impressions_diff'] = result_df['feb_impressions'] - result_df['jan_impressions']
result_df['ctr_diff'] = result_df['feb_ctr'] - result_df['jan_ctr']
result_df['position_diff'] = result_df['jan_position'] - result_df['feb_position']  # Note: Reversed to make positive = improvement

# Calculate percentage changes
result_df['clicks_pct_change'] = result_df.apply(
    lambda x: ((x['feb_clicks'] - x['jan_clicks']) / x['jan_clicks'] * 100) if x['jan_clicks'] > 0 else float('inf'),
    axis=1
)
result_df['impressions_pct_change'] = result_df.apply(
    lambda x: ((x['feb_impressions'] - x['jan_impressions']) / x['jan_impressions'] * 100) if x['jan_impressions'] > 0 else float('inf'),
    axis=1
)
result_df['position_pct_change'] = result_df.apply(
    lambda x: ((x['jan_position'] - x['feb_position']) / x['jan_position'] * 100) if x['jan_position'] > 0 else float('inf'),
    axis=1
)

# Round all numeric columns to 2 decimal places
numeric_cols = result_df.select_dtypes(include=['float64', 'int64']).columns
result_df[numeric_cols] = result_df[numeric_cols].round(2)

# Replace infinity values with None for cleaner output
result_df.replace([float('inf'), -float('inf')], None, inplace=True)

# Sort by the absolute difference in clicks (highest impact first)
result_df = result_df.sort_values(by='clicks_diff', ascending=False)

# Save the result
result_df.to_csv('data/category_compariso-ngsc-category-t9n.csv', index=False)

print(f"Analysis completed. Found {len(result_df)} unique content types.")
print("\nTop 5 categories with biggest click increases:")
print(result_df.head(5)[['content_type', 'clicks_diff', 'clicks_pct_change']])

print("\nTop 5 categories with biggest position improvements:")
print(result_df.sort_values(by='position_diff', ascending=False).head(5)[['content_type', 'jan_position', 'feb_position', 'position_diff']])

print("\nTop 5 categories with biggest click decreases:")
print(result_df.sort_values(by='clicks_diff')[['content_type', 'clicks_diff', 'clicks_pct_change']].head(5))

Analysis completed. Found 11 unique content types.

Top 5 categories with biggest click increases:
                       content_type  clicks_diff  clicks_pct_change
8              Not_reviewed_brokers       6113.0             403.50
9                          Homepage         51.0              16.24
10                            Other        -38.0             -33.93
7   Company_name_options_megascaled       -496.0             -22.85
5                         Education      -1093.0             -16.06

Top 5 categories with biggest position improvements:
                             content_type  jan_position  feb_position  \
4                                   Tools          8.92          5.76   
3   How_to_buy_shares_and_ETFs_megascaled         20.27         19.01   
1                           Broker_review         13.09         13.33   
10                                  Other         32.74         33.16   
0                       Safety_megascaled         11.38         11.87   

